# SPLトークンをミントして NFT を作成する

Metaplex Python API を利用してNFTを作成する

- [Google Colab で実行するためのリンク](https://colab.research.google.com/github/regonn/Solana-NFT/blob/master/SolanaNFT_04.ipynb)

- [Softgateさんの NFT 技術基礎論](https://quirky-caution-3c5.notion.site/NFT-507d8038e2cd46de831e42ba2b4c0a90)
    - 「3. SPL トークンをミント」
    - 「4. createMetadataAccount でメタデータアカウントを作成」
    - 「5. createMasterEdition で NFT を作成」
- Metaplex Python API (https://github.com/metaplex-foundation/python-api)
    - deploy コマンド
        - 「3. SPL トークンをミント」
    - mint コマンド
        - 「4. createMetadataAccount でメタデータアカウントを作成」
        - 「5. createMasterEdition で NFT を作成」

## 環境構築

In [ ]:
!git clone https://github.com/metaplex-foundation/python-api.git

In [ ]:
%cd python-api

In [ ]:
# ライブラリでバージョン管理されていないので、動画撮影時の最新コミットに固定
!git checkout ce1b314b629270c2d357b5698afd7ccbf3308a41

In [ ]:
!pip install -r ./requirements.txt

In [ ]:
from api.metaplex_api import MetaplexAPI
from cryptography.fernet import Fernet

# Account は非推奨(deprecated) だが、APIが Account の時に書かれているコードなのでこっちを利用している
from solana.account import Account
# Solana 0.9.2 だと Keypair は実装されていないので注意
# from solana.keypair import Keypair
from solana.rpc.api import Client
import base58
import json

In [ ]:
# 前回作った Arweave でのオフチェーンメタデータのURI
offchain_metadata_uri = 'https://arweave.net/s9IU4ite53UwvThJc4gqZFXCSJ23jUF5e0PEkwo1SwY/'

In [ ]:
# 今回は devnet で行う
api_endpoint = "https://api.devnet.solana.com"

In [ ]:
# 秘密鍵情報等を Dictionary で管理しておいて、最後に JSON で出力するようにするための変数
keys_dict = {}

In [ ]:
keys_dict['api_endpoint'] = api_endpoint

In [ ]:
# 暗号化用のキーを生成
keys_dict["descryption_key"] = Fernet.generate_key().decode("ascii")

In [ ]:
# トークン発行等を行うアカウント(Wallet)を作成
source_account = Account()

In [ ]:
# ここらへんも、Account の場合は account.public_key() だけど、Keypair の場合は account.public_key で取得時の書き方が変わってくるので注意
keys_dict["source_account_secret_key"] = list(source_account.secret_key())[:32]
keys_dict["source_account_public_key"] = str(source_account.public_key())

In [ ]:
source_account.public_key()

In [ ]:
# シークレットキーからAccountを復元して、public_key が同じことを確認
Account(keys_dict["source_account_secret_key"]).public_key()

In [ ]:
# メインネットでやる場合は作ったWalletにSOLを送金しておく必要がある
# devnet などでは https://solfaucet.com/ とかで Airdrop を申請して、solscan で残高を確認しておくと良い

In [ ]:
# Metaplex に渡す用のコンフィグ生成
metaplex_config_dict = {
    "PRIVATE_KEY": base58.b58encode(source_account.secret_key()).decode("ascii"),
    "PUBLIC_KEY": str(source_account.public_key()),
    "DECRYPTION_KEY": keys_dict["descryption_key"]
}

In [ ]:
metaplex_api = MetaplexAPI(metaplex_config_dict)

In [ ]:
result_json = metaplex_api.deploy(api_endpoint, "Bubbles", "BUBBLENFT")

In [ ]:
mint_address = json.loads(result_json)['contract']

In [ ]:
f'https://solscan.io/token/{mint_address}?cluster=devnet'

In [ ]:
keys_dict['mint_address'] = mint_address

In [ ]:
# NFTを送る先のwalletを作成している。本番だったらWalletの public key だけあれば大丈夫
wallet_json = metaplex_api.wallet()

In [ ]:
wallet = json.loads(wallet_json)

In [ ]:
wallet['address']

In [ ]:
# PrivateKey も含まれているが表示しない方がいいので、public_key で値が正しいことを確認
Account(wallet['private_key']).public_key()

In [ ]:
keys_dict['wallet_private_key'] = wallet['private_key']
keys_dict['wallet_public_key'] = wallet['address']

In [ ]:
# TOKEN 受け取り用に wallet へ少額の SOL を送る、これも本番で別途walletがあるなら必要ない
metaplex_api.topup(api_endpoint, wallet['address'])

In [ ]:
# MINT を行う
metaplex_api.mint(api_endpoint, mint_address, wallet['address'], offchain_metadata_uri)

In [ ]:
with open('../solana-nft-keys.json', 'w') as fp:
    json.dump(keys_dict, fp)

In [ ]:
# 第2回で利用したコード

from solana.publickey import PublicKey
import base64
import struct
import requests
METADATA_PROGRAM_ID = PublicKey('metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s')

# metadataアカウントを取得する
def get_metadata_account(mint_key):
    return PublicKey.find_program_address(
        [b'metadata', bytes(METADATA_PROGRAM_ID), bytes(PublicKey(mint_key))],
        METADATA_PROGRAM_ID
    )[0]

# バイナリデータからmetadata情報を取り出す
def unpack_metadata_account(data):
    assert(data[0] == 4)
    i = 1
    source_account = base58.b58encode(bytes(struct.unpack('<' + "B"*32, data[i:i+32])))
    i += 32
    mint_account = base58.b58encode(bytes(struct.unpack('<' + "B"*32, data[i:i+32])))
    i += 32
    name_len = struct.unpack('<I', data[i:i+4])[0]
    i += 4
    name = struct.unpack('<' + "B"*name_len, data[i:i+name_len])
    i += name_len
    symbol_len = struct.unpack('<I', data[i:i+4])[0]
    i += 4 
    symbol = struct.unpack('<' + "B"*symbol_len, data[i:i+symbol_len])
    i += symbol_len
    uri_len = struct.unpack('<I', data[i:i+4])[0]
    i += 4 
    uri = struct.unpack('<' + "B"*uri_len, data[i:i+uri_len])
    i += uri_len
    fee = struct.unpack('<h', data[i:i+2])[0]
    i += 2
    has_creator = data[i] 
    i += 1
    creators = []
    verified = []
    share = []
    if has_creator:
        creator_len = struct.unpack('<I', data[i:i+4])[0]
        i += 4
        for _ in range(creator_len):
            creator = base58.b58encode(bytes(struct.unpack('<' + "B"*32, data[i:i+32])))
            creators.append(creator)
            i += 32
            verified.append(data[i])
            i += 1
            share.append(data[i])
            i += 1
    primary_sale_happened = bool(data[i])
    i += 1
    is_mutable = bool(data[i])
    metadata = {
        "update_authority": source_account,
        "mint": mint_account,
        "data": {
            "name": bytes(name).decode("utf-8").strip("\x00"),
            "symbol": bytes(symbol).decode("utf-8").strip("\x00"),
            "uri": bytes(uri).decode("utf-8").strip("\x00"),
            "seller_fee_basis_points": fee,
            "creators": creators,
            "verified": verified,
            "share": share,
        },
        "primary_sale_happened": primary_sale_happened,
        "is_mutable": is_mutable,
    }
    return metadata

In [ ]:
metadata_account = get_metadata_account(mint_address)
decoded_data = base64.b64decode(Client(api_endpoint).get_account_info(metadata_account)['result']['value']['data'][0])
metadata = unpack_metadata_account(decoded_data)

In [ ]:
metadata

In [ ]:
metadata_uri = metadata['data']['uri']
response = requests.get(metadata_uri)
response_json = response.json()

In [ ]:
response_json

In [ ]:
f'https://solscan.io/account/{wallet["address"]}?cluster=devnet'